In [ ]:
import numpy
# https://en.wikipedia.org/wiki/PNG
# https://github.com/miloyip/svpng/blob/master/svpng.inc
def display(A):
    if A.ndim != 3 and A.ndim != 2: raise ValueError("A.ndim != 3 and A.ndim != 2")
    A = A.astype(numpy.uint8)
    import PIL.Image
    return PIL.Image.fromarray(A)
def check_img1chan(image):
    if image.ndim != 2: raise ValueError("image.ndim != 2")
def check_img3chan(image):
    if image.ndim != 3: return ValueError("image.ndim != 3")
    if image.shape[2] != 3: return ValueError("image.shape[2] != 3")

In [ ]:
A = numpy.load('lena_gray.npy')
display(A)

In [ ]:
# Cropping & downsize (naive)
collage = numpy.concatenate((A[128:384,128:384], A[0:512:2,0:512:2]), axis=1)
display(collage)

In [ ]:
# Simple rotations
D = A[0:512:2,0:512:2]
def f0(image): check_img1chan(image); return numpy.flip(image, axis=0)
def f1(image): check_img1chan(image); return numpy.flip(image, axis=1)
collage1 = numpy.concatenate((D,     D.T,     f0(D),     f0(D).T),     axis=1)
collage2 = numpy.concatenate((f1(D), f1(D).T, f0(f1(D)), f0(f1(D)).T), axis=1)
collage = numpy.concatenate((collage1, collage2), axis=0)
display(collage)

In [ ]:
# Upsize (naive)
canvas = numpy.zeros((512, 512)).astype(numpy.uint8)
canvas[0:512:2,0:512:2] = A[0:512:2,0:512:2]
display(canvas)

In [ ]:
# Resize

# https://chao-ji.github.io/jekyll/update/2018/07/19/BilinearResize.html
def bilinear_resize(image, shape):
    check_img1chan(image)

    image_height, image_width = image.shape
    new_height, new_width = shape
    image = image.ravel()
    ratio_y = (image_height-1)/(new_height-1)
    ratio_x = (image_width-1)/(new_width-1)
    indices_y, indices_x = numpy.divmod(numpy.arange(new_height*new_width), new_width)
    scaled_indices_y = ratio_y*indices_y
    scaled_indices_x = ratio_x*indices_x
    lows_y = numpy.floor(scaled_indices_y).astype(numpy.int64)
    highs_y = numpy.ceil(scaled_indices_y).astype(numpy.int64)
    lows_x = numpy.floor(scaled_indices_x).astype(numpy.int64)
    highs_x = numpy.ceil(scaled_indices_x).astype(numpy.int64)
    weights_y = ratio_y*indices_y - lows_y
    weights_x = ratio_x*indices_x - lows_x
    a = image[lows_y * image_width + lows_x]
    b = image[lows_y * image_width + highs_x]
    c = image[highs_y * image_width + lows_x]
    d = image[highs_y * image_width + highs_x]
    new_image = (
          a * (1-weights_x) * (1-weights_y)
        + b * weights_x * (1-weights_y)
        + c * (1-weights_x) * weights_y
        + d * weights_x * weights_y
    )
    return new_image.reshape(new_height, new_width)
display(bilinear_resize(A, (600,400)))

In [ ]:
# TODO: can I also interpolate to fill the gaps?
def coordinates_transform(image, matrix):
    check_img1chan(image)
    if matrix.shape != (2, 2): raise ValueError("image.ndim != 2,2")

    image_height, image_width = image.shape
    image_coords = numpy.stack(numpy.divmod(numpy.arange(image_height*image_width), image_width)).T
    image_coords_centered = image_coords - numpy.array(image.shape)//2
    image_coords_transformed = image_coords_centered@matrix.T

    max_index = numpy.argmax(numpy.linalg.norm(image_coords_transformed, axis=0))
    canvas_shape = numpy.ceil(numpy.max(numpy.abs(image_coords_transformed), axis=0)).astype(numpy.int64)*2

    canvas = numpy.zeros(canvas_shape)
    image_coords_canvas = image_coords_transformed.astype(numpy.int64) + canvas_shape//2
    canvas[image_coords_canvas[:,0], image_coords_canvas[:,1]] = image.ravel()
    return canvas.reshape(canvas_shape)

# https://en.wikipedia.org/wiki/Transformation_matrix#Examples_in_2_dimensions
# http://datagenetics.com/blog/august32013/index.html
angle = 30.
cos_theta = numpy.cos(angle)
sin_theta = numpy.sin(angle)
rot = numpy.array([
    [ cos_theta, sin_theta],
    [-sin_theta, cos_theta]
])
identity = numpy.eye(2)
stretch = numpy.array([[2,0],[0,1]])

# TODO: affine transformations (e.g. perspective)
# TODO: alpha compositing

display(coordinates_transform(A, rot))

In [ ]:
def transform_invert(image): return 1 - image
def transform_gamma(image): return image**0.4
def transform_threshold(image): return image > 0.5
# https://en.wikipedia.org/wiki/Posterization
# https://en.wikipedia.org/wiki/Quantization_(image_processing)
# https://lettier.github.io/3d-game-shaders-for-beginners/posterization.html
def transform_discretize(image): return numpy.floor(image*10)/10

def plot_function(f):
    x = numpy.linspace(0,1,256)
    y = f(x)
    plot = y.reshape(-1, 1) < x
    plot = numpy.flip(plot, axis=1).T
    plot = plot*255
    return plot

display(numpy.concatenate(
    (plot_function(transform_invert),
     plot_function(transform_gamma),
     plot_function(transform_threshold),
     plot_function(transform_discretize),
    ), axis=1))

In [ ]:
def intensity_transform(image, transform):
    check_img1chan(image)
    image_scaled = image/255
    image_trans = transform(image_scaled)
    return (image_trans*255)
    return res

display(numpy.concatenate(
    (intensity_transform(A, transform_invert),
     intensity_transform(A, transform_gamma),
     intensity_transform(A, transform_threshold),
     intensity_transform(A, transform_discretize),
    ), axis=1))

In [ ]:
# Histogram
# https://vincmazet.github.io/bip/histogram/transformations.html
# https://www.dsi.unive.it/~bergamasco/teachingfiles/cvslides/2_intensity_transformations.pdf
def image_histogram(image):
    check_img1chan(image)
    hist, _ = numpy.histogram(image, 256)
    if hist.ndim != 1: raise ValueError("hist.ndim != 1:")
    plot = hist.reshape(-1, 1)/hist.max() < numpy.linspace(0.,1.,256)
    plot = numpy.flip(plot, axis=1).T
    plot = plot*255
    return plot
display(image_histogram(A))

In [ ]:
def histogram_equalization(image):
    check_img1chan(image)
    hist, _ = numpy.histogram(image, 256)
    cdf = numpy.cumsum(hist)
    return (cdf[image]/image.size*255)
display(numpy.concatenate((A, histogram_equalization(A)), axis=1))

In [ ]:
display(numpy.concatenate((image_histogram(A), image_histogram(histogram_equalization(A))), axis=1))

In [ ]:
# https://stackoverflow.com/questions/43086557/convolve2d-just-by-using-numpy
# TODO: this should take kernel size and a function as argument.
def conv2d(image, filt, median=False): # aka local_transform
    check_img1chan(image)

    if filt.ndim != 2: raise ValueError("filt.ndim != 2")
    if filt.shape[0] != filt.shape[1]: raise ValueError("filt.shape[0] != filt.shape[1]")
    if filt.shape[0]%2 == 0 or filt.shape[1]%2 == 0: raise ValueError("filt.shape[0]%2 == 0 or filt.shape[1]%2 == 0")

    pad = (filt.shape[0]-1)//2
    image = numpy.pad(image, (pad,pad), mode='edge')
    # TODO: s = tuple(numpy.subtract(image.shape, filt.shape) + 1) + filt.shape i.e. (512, 512, 3, 3)
    s = filt.shape + tuple(numpy.subtract(image.shape, filt.shape) + 1)
    # TODO: can't this be just a sliding window view?
    subM = numpy.lib.stride_tricks.as_strided(image, shape = s, strides = image.strides * 2)
    res = subM.reshape(filt.shape[0], filt.shape[1], (image.shape[0]-pad*2)*(image.shape[1]-pad*2))
    if not median:
        res = (res.transpose((2,0,1))*filt).transpose(1,2,0)
        res = numpy.sum(res.reshape(filt.shape[0]*filt.shape[1], (image.shape[0]-pad*2)*(image.shape[1]-pad*2)), axis=0)
    else:
        res = numpy.median(res.reshape(filt.shape[0]*filt.shape[1], (image.shape[0]-pad*2)*(image.shape[1]-pad*2)), axis=0)
    res = res.reshape(image.shape[0]-pad*2, image.shape[1]-pad*2)
    # return numpy.einsum('ij,ijkl->kl', filt, subM)
    return res

def box_kernel(l):
    kernel = numpy.ones((l, l))
    kernel /= l*l
    return kernel

#https://stackoverflow.com/a/43346070
def gaussian_kernel(l, sigma=1.):
    ax = numpy.linspace(-(l - 1) / 2, (l - 1) / 2, l)
    gauss = numpy.exp(-0.5 * ax**2 / sigma**2)
    kernel = numpy.outer(gauss, gauss)
    kernel /= numpy.sum(kernel)
    return kernel

# TODO: https://www.cs.uoi.gr/~cnikou/Courses/Image_Analysis/01_Linear_Filters.pdf (cool demos)
# https://www.cs.toronto.edu/~jepson/csc420/notes/linearFiltering.pdf
# https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
display(numpy.concatenate((A, conv2d(A, box_kernel(3)), conv2d(A, gaussian_kernel(9)), conv2d(A, numpy.empty((5,5)), True)), axis=1))

In [ ]:
# Edge detection

# https://en.wikipedia.org/wiki/Sobel_operator
def sobel(image):
    check_img1chan(image)
    Sx = numpy.array(((+1,0,-1),
                      (+2,0,-2),
                      (+1,0,-1)))
    Sy = Sx.T
    image = image.astype(numpy.float64)
    Gx = conv2d(image, Sx)
    Gy = conv2d(image, Sy)
    G = numpy.hypot(Gx, Gy)
    Theta = numpy.arctan2(Gx, Gy)
    return G
# https://en.wikipedia.org/wiki/Prewitt_operator
def prewit(image):
    check_img1chan(image)
    Sx = numpy.array(((+1,0,-1),
                      (+1,0,-1),
                      (+1,0,-1)))
    Sy = Sx.T
    image = image.astype(numpy.float64)
    Gx = conv2d(image, Sx)
    Gy = conv2d(image, Sy)
    G = numpy.hypot(Gx, Gy)
    Theta = numpy.arctan2(Gx, Gy)
    return G
# https://en.wikipedia.org/wiki/Roberts_cross
def roberts(image):
    check_img1chan(image)
    # FIXME: requires support for filters with side length even.
    Sx = numpy.array(((+1, 0),
                      ( 0,-1)))
    Sy = numpy.array(((0, +1),
                      (-1, 0)))
    image = image.astype(numpy.float64)
    Gx = conv2d(image, Sx)
    Gy = conv2d(image, Sy)
    G = numpy.hypot(Gx, Gy)
    Theta = numpy.arctan2(Gx, Gy)
    return G

# TODO: map theta to color in HSI image...
# https://en.wikipedia.org/wiki/Canny_edge_detector is a much more complicated algorithm...
display(numpy.concatenate((sobel(A), prewit(A)), axis=1))

In [ ]:
A = numpy.load('lena_std.npy')
# https://tannerhelland.com/2011/10/01/grayscale-image-algorithm-vb6.html
def channels_transform(A, T):
    check_img3chan(A)
    A = A/255
    A = A@T
    A = A*255
    return A
luminance = numpy.array((0.2126,0.7152,0.0722))
average = numpy.array((1/3,1/3,1/3))

# https://leware.net/photo/blogSepia.html
sepia = numpy.array((
    (0.393, 0.349, 0.272),
    (0.769, 0.686, 0.534),
    (0.189, 0.168, 0.131),
))

B = channels_transform(A, luminance)
B = numpy.stack((B, B, B), axis=2)
C = A[:,:,[2,1,0]] # BGR (just a permutation matrix)
D = numpy.clip(channels_transform(A, sepia), 0, 255)
display(numpy.concatenate((A,B,C, D), axis=1))
# TODO:
# https://lisyarus.github.io/blog/posts/transforming-colors-with-matrices.html
# https://www.sfu.ca/~jtmulhol/py4math/linalg/ap-image-basics/
# https://www.imatest.com/docs/colormatrix/

# Other filters http://www.jhlabs.com/ip/filters/index.html

In [ ]:
# https://mattlockyer.github.io/iat455/documents/rgb-hsv.pdf
# https://faculty.kfupm.edu.sa/ics/lahouari/Teaching/colorspacetransform-1.0.pdf
def rgb2hsv(image):
    check_img3chan(image)

    image_scaled = image/255
    R = image_scaled[:,:,0]
    G = image_scaled[:,:,1]
    B = image_scaled[:,:,2]
    
    ms_max = numpy.argmax(image_scaled, axis=2)
    ms_min = numpy.argmin(image_scaled, axis=2)
    deltas = image_scaled.reshape(-1, 3)[0,ms_max.ravel()] - image_scaled.reshape(-1, 3)[0,ms_min.ravel()]
    deltas = deltas.reshape(image.shape[0], image.shape[1])
    if numpy.isclose(deltas, 0.0).any(): return ValueError("numpy.isclose(delta, 0.0).any()")

    hue = numpy.stack((G-B, B-R, R-G), axis=2).transpose((2,0,1))/deltas
    hue = hue.transpose((1,2,0)) + numpy.array((0.,2.,4.))
    hue = hue.reshape(-1,3)[0,ms_max].reshape(image.shape[0], image.shape[1])
    hue = (hue*255).astype(numpy.uint8)

    value = (ms_max*255)

    saturation = numpy.divide(deltas, value, out=numpy.zeros_like(deltas), where=value!=0)
    saturation = (saturation*255).astype(numpy.uint8)
    return numpy.stack((hue, saturation, value), axis=2)

def hsv2rgb(image):
    check_img3chan(image)

    H = image[:,:,0]%6
    S = image[:,:,1]/255
    V = image[:,:,2]/255

    alphas = V*(1-S)
    beta = 0
    gamma = 0
    
    pass

B = rgb2hsv(A)
B[:,:,0]%6

In [ ]:
# Laplacian pyramid https://www.youtube.com/watch?v=U7qa7i0K9C4
# Anti-aliasing (signal sampling) https://www.youtube.com/watch?v=fTJjPGaPsq4
# Template matching with cross-correlation (no SIFT) https://www.youtube.com/watch?v=EO1-MCWfXCU
# Blur filter and convoltion theorem https://www.youtube.com/watch?v=xvDeFABiwrg